# Collecting the Public Comments Regarding the EPA's Revised Definition of "Waters of the United States" (WOTUS)

The Environmental Protection Agency (EPA) and the the Department of the Army conducted a reevaluation and revision of the definition of "Waters of the United States" under an executive order from President Trump. Their proposed rule redefines the scope of waters federally regulated under the Clean Water Act (CWA). This rule was open for public comment until April 15, 2019 at [regulations.gov](https://www.regulations.gov/docket?D=EPA-HQ-OW-2018-0149).

As of October 2019, there were 625,521 comments received on the docket, of which 11,412 are posted and publicly available under "public submissions". The discrepancy can be attributed to the following note on the site:

>Agencies review all submissions, however some agencies may choose to redact, or withhold, certain submissions (or portions thereof) such as those containing private or proprietary information, inappropriate language, or duplicate/near duplicate examples of a mass-mail campaign.

The regulations.gov site actually has an API to request data programmatically, however API keys are only available to organizations (not individuals). Thus, this project uses a web scraping program to loop over a `.csv` file with a selection of docket information (including unique pages for each public submission) to collect and compile the comments. The original `.csv` file  is saved in the `/Data` directory, or it can be exported directly from the page [to view public comments](https://www.regulations.gov/docketBrowser?rpp=25&so=DESC&sb=commentDueDate&po=0&dct=PS&D=EPA-HQ-OW-2018-0149), the scraper uses a simplified version with comment IDs and URLs only. This file is generated below.

There are two noteworthy items about the web scraper. First, the regulations.gov site is rendered using JavaScript, so the web scraper needs to use the [Selenium package](https://github.com/SeleniumHQ/selenium/) to open a browser, pause to let the page load, then grab the comment. Second, not all comments are available as text on the site, since it allows public submissions in attachment form as well as typing them in directly. Approximately 28% of the comments are in attachment form (most of which are `pdf` format), and the scraper could not collect the information within.

## Library Imports

In [2]:
import glob
import sys
import sqlite3
import warnings
warnings.filterwarnings('ignore') 

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)

## Sanitize Docket File for Web Scraper Use

This section imports the original docket information file, removes comments in attachment form and saves only the necessary columns (Document ID and Document Detail) for the web scraper.

In [34]:
docket = pd.read_csv('./Data/DOCKET_EPA-HQ-OW-2018-0149.csv', header=2)

docket.head()

,Document Title,Document Type,Attachment Count,Submitter Last Name,Submitter First Name,Organization,Document ID,Posted Date,Received Date,Comment Start Date,Comment Due Date,Implementation Date,Effective Date,Related RINs,Document SubType,Subject,Abstract,Status,Source Citation,OMB Approval Number,FR Citation,Federal Register Number,Start End Page,Special Instructions,Legacy ID,Post Mark Date,File Type,Number Of Pages,Paper Width,Paper Length,Exhibit Type,Exhibit Location,Document Field_1,Document Field_2,Document Detail
0,Anonymous public comment,PUBLIC SUBMISSIONS,NaN,NaN,NaN,NaN,EPA-HQ-OW-2018-0149-6689,04/29/2019,04/11/2019,NaN,NaN,NaN,NaN,NaN,Public Comment,NaN,NaN,Posted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04/11/2019,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,https://www.regulations.gov/document?D=EPA-HQ-...
1,Anonymous public comment,PUBLIC SUBMISSIONS,NaN,NaN,NaN,NaN,EPA-HQ-OW-2018-0149-6688,04/29/2019,04/11/2019,NaN,NaN,NaN,NaN,NaN,Public Comment,NaN,NaN,Posted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04/11/2019,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,https://www.regulations.gov/document?D=EPA-HQ-...
2,"Comment submitted by Mike Hook, Craighead Coun...",PUBLIC SUBMISSIONS,1.0,NaN,NaN,NaN,EPA-HQ-OW-2018-0149-6690,04/29/2019,04/11/2019,NaN,NaN,NaN,NaN,NaN,Government Local,NaN,NaN,Posted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04/11/2019,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,https://www.regulations.gov/document?D=EPA-HQ-...
3,Anonymous public comment,PUBLIC SUBMISSIONS,NaN,NaN,NaN,NaN,EPA-HQ-OW-2018-0149-7377,05/01/2019,04/11/2019,NaN,NaN,NaN,NaN,NaN,Public Comment,NaN,NaN,Posted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04/11/2019,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,https://www.regulations.gov/document?D=EPA-HQ-...
4,"Comment submitted by Anne Hulick, CT Director,...",PUBLIC SUBMISSIONS,1.0,NaN,NaN,NaN,EPA-HQ-OW-2018-0149-8930,05/04/2019,04/15/2019,NaN,NaN,NaN,NaN,NaN,Company/Organization Comment,NaN,NaN,Posted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04/15/2019,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,https://www.regulations.gov/document?D=EPA-HQ-...


In [35]:
docket.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11412 entries, 0 to 11411
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Document Title           11412 non-null  object 
 1   Document Type            11412 non-null  object 
 2   Attachment Count         3224 non-null   float64
 3   Submitter Last Name      0 non-null      float64
 4   Submitter First Name     0 non-null      float64
 5   Organization             0 non-null      float64
 6   Document ID              11411 non-null  object 
 7   Posted Date              11411 non-null  object 
 8   Received Date            11411 non-null  object 
 9   Comment Start Date       0 non-null      float64
 10  Comment Due Date         0 non-null      float64
 11  Implementation Date      0 non-null      float64
 12  Effective Date           0 non-null      float64
 13  Related RINs             0 non-null      float64
 14  Document SubType      

In [36]:
# Drop columns with which have all null values
docket.dropna(axis=1, how='all', inplace=True)

# Drop row with withdrawn comment (almost all nulls)
docket.dropna(thresh=6, inplace=True)

docket.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11411 entries, 0 to 11411
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Document Title    11411 non-null  object 
 1   Document Type     11411 non-null  object 
 2   Attachment Count  3224 non-null   float64
 3   Document ID       11411 non-null  object 
 4   Posted Date       11411 non-null  object 
 5   Received Date     11411 non-null  object 
 6   Document SubType  11409 non-null  object 
 7   Abstract          72 non-null     object 
 8   Status            11411 non-null  object 
 9   Post Mark Date    11353 non-null  object 
 10  File Type         50 non-null     object 
 11  Number Of Pages   11411 non-null  float64
 12  Document Detail   11411 non-null  object 
dtypes: float64(2), object(11)
memory usage: 1.2+ MB


In [37]:
# Covert date columns to datetime format
for col in ['Posted Date', 'Received Date', 'Post Mark Date']:
    docket[col] = pd.to_datetime(docket[col], format='%m/%d/%Y')

docket.dtypes

Document Title              object
Document Type               object
Attachment Count           float64
Document ID                 object
Posted Date         datetime64[ns]
Received Date       datetime64[ns]
Document SubType            object
Abstract                    object
Status                      object
Post Mark Date              object
File Type                   object
Number Of Pages            float64
Document Detail             object
dtype: object

In [38]:
docket.head()

,Document Title,Document Type,Attachment Count,Document ID,Posted Date,Received Date,Document SubType,Abstract,Status,Post Mark Date,File Type,Number Of Pages,Document Detail
0,Anonymous public comment,PUBLIC SUBMISSIONS,NaN,EPA-HQ-OW-2018-0149-6689,2019-04-29,2019-04-11,Public Comment,NaN,Posted,04/11/2019,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...
1,Anonymous public comment,PUBLIC SUBMISSIONS,NaN,EPA-HQ-OW-2018-0149-6688,2019-04-29,2019-04-11,Public Comment,NaN,Posted,04/11/2019,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...
2,"Comment submitted by Mike Hook, Craighead Coun...",PUBLIC SUBMISSIONS,1.0,EPA-HQ-OW-2018-0149-6690,2019-04-29,2019-04-11,Government Local,NaN,Posted,04/11/2019,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...
3,Anonymous public comment,PUBLIC SUBMISSIONS,NaN,EPA-HQ-OW-2018-0149-7377,2019-05-01,2019-04-11,Public Comment,NaN,Posted,04/11/2019,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...
4,"Comment submitted by Anne Hulick, CT Director,...",PUBLIC SUBMISSIONS,1.0,EPA-HQ-OW-2018-0149-8930,2019-05-04,2019-04-15,Company/Organization Comment,NaN,Posted,04/15/2019,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...


In [39]:
docket['Attachment Count'].value_counts()

1.0     2913
2.0      264
3.0       12
4.0        8
9.0        5
19.0       4
6.0        3
20.0       3
11.0       3
12.0       3
10.0       2
5.0        1
17.0       1
7.0        1
16.0       1
Name: Attachment Count, dtype: int64

In [40]:
docket['File Type'].value_counts()

pdf     35
jpg     10
png      3
docx     2
Name: File Type, dtype: int64

In [9]:
# Simplify DataFrame, save as file for scraper

# Remove comments where it's imbedded in an attachment
scraper = docket[docket['Attachment Count'].isnull()]

# Save only critical info for scraper
scraper = scraper[['Document ID', 'Document Detail']]

print('Total comments posted directly to site: {0:,.0f}'.format(len(scraper)))

# Uncomment next line to generate the simplified scraper CSV file
# scraper.to_csv('./Data/scraper_csv.csv', index=False)

Total comments posted directly to site: 8,187


## Create DataFrame with Comments

This section builds the full DataFrame of docket information with the text of each comment and the sample of labels. It imports all comments into separate DataFrame objects, concatenates them to create one DF of all IDs and comment text, then merges it with the `docket` DataFrame to combine the comments with other metadata. There were 41 files with blanks for the comment that needed to be manually copied in. Finally, another join is performed to add the labels.

This object is then serialized and saved to avoid this step going forward.

Note: this code assumes you have run the text file web scraper (`scraper_text.py` - see README.md for instructions) to collect the comments - otherwise the `./Data/Comments/` directory is empty.

In [10]:
# Collect a list of all comment files (saved as .txt format)
comment_files = glob.glob('./Data/Comments/*.txt')
len(comment_files)

8187

In [11]:
# Create DF for each comment, concatenate into one DF
# Uncomment code to build the comment DF from the scraped files

# dfs = []

# for cf in comment_files:
#     try:
#         tmp = pd.read_csv(cf, sep=r'\n', header=None)
#         tmp.index = ['Document ID', 'Comment']
#         tmp = tmp.transpose()
#         dfs.append(tmp)
#     except ValueError as e:
#         print(cf)
#         continue


# com_df = pd.concat(dfs, ignore_index=True)

# com_df.head()

In [12]:
# Serialize com_df to pickled object
# com_df.to_pickle('./Data/comments_df.pkl')

In [3]:
# Read serialized DF vs re-building it
com_df = pd.read_pickle('./Data/comments_df.pkl')

com_df.head()

,Document ID,Comment
0,EPA-HQ-OW-2018-0149-8815,EPAs proposal would result in a profound weake...
1,EPA-HQ-OW-2018-0149-3940,I am commenting to urge that this new regulati...
2,EPA-HQ-OW-2018-0149-3798,It is absolutely laughable to pretend in regul...
3,EPA-HQ-OW-2018-0149-6186,I strongly oppose the EPAs proposed replacemen...
4,EPA-HQ-OW-2018-0149-7298,I write to express my opposition to the Enviro...


In [4]:
com_df.shape

(8187, 2)

In [15]:
# Merge docket DF with comments DF
# Uncomment code to perform the merge directly
# full_df = pd.merge(left=docket, right=com_df, how='right', on='Document ID')

# full_df.head()

In [16]:
# Remove mostly/all-null and columns with all same values
# cols_to_drop = ['Document Type', 'Attachment Count', 'Abstract', 'Status', 'File Type', 'Number Of Pages']

# full_df.drop(cols_to_drop, axis=1, inplace=True)

In [17]:
# Serialize full DF as pickle object
# full_df.to_pickle('./Data/full_df.pkl')

In [5]:
# Test pickle load
full_df = pd.read_pickle('./Data/full_df.pkl')

full_df.head()

,Document Title,Document ID,Posted Date,Received Date,Document SubType,Post Mark Date,Document Detail,Comment
0,Anonymous public comment,EPA-HQ-OW-2018-0149-6689,2019-04-29,2019-04-11,Public Comment,04/11/2019,https://www.regulations.gov/document?D=EPA-HQ-...,I write to express my opposition to the Enviro...
1,Anonymous public comment,EPA-HQ-OW-2018-0149-6688,2019-04-29,2019-04-11,Public Comment,04/11/2019,https://www.regulations.gov/document?D=EPA-HQ-...,I write to express my opposition to the Enviro...
2,Anonymous public comment,EPA-HQ-OW-2018-0149-7377,2019-05-01,2019-04-11,Public Comment,04/11/2019,https://www.regulations.gov/document?D=EPA-HQ-...,Please abandon this proposed rollback of the 2...
3,Anonymous public comment,EPA-HQ-OW-2018-0149-6707,2019-04-29,2019-04-11,Public Comment,04/11/2019,https://www.regulations.gov/document?D=EPA-HQ-...,"My husband and I have two wonderful daughters,..."
4,Comment submitted by T. Dahlin,EPA-HQ-OW-2018-0149-7983,2019-05-01,2019-04-12,Public Comment,04/12/2019,https://www.regulations.gov/document?D=EPA-HQ-...,Please do not change the definitions within th...


In [6]:
full_df.shape

(8187, 8)

In [7]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8187 entries, 0 to 8186
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Document Title    8187 non-null   object        
 1   Document ID       8187 non-null   object        
 2   Posted Date       8187 non-null   datetime64[ns]
 3   Received Date     8187 non-null   datetime64[ns]
 4   Document SubType  8187 non-null   object        
 5   Post Mark Date    8134 non-null   object        
 6   Document Detail   8187 non-null   object        
 7   Comment           8187 non-null   object        
dtypes: datetime64[ns](2), object(6)
memory usage: 575.6+ KB


In [8]:
# Read in labeled sample
labels = pd.read_excel('./Data/comments_labeled.xlsx', index_col=0)
labels.head()

,Comment,Support_Rule_Change
0,I write to express my opposition to the Enviro...,0.0
1,I write to express my opposition to the Enviro...,0.0
2,Please abandon this proposed rollback of the 2...,0.0
3,"My husband and I have two wonderful daughters,...",0.0
4,Please do not change the definitions within th...,0.0


In [9]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8187 entries, 0 to 8186
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Comment              8187 non-null   object 
 1   Support_Rule_Change  1221 non-null   float64
dtypes: float64(1), object(1)
memory usage: 191.9+ KB


In [10]:
# Merge labels into full DF
full_labeled = full_df.join(labels, rsuffix='_DROP')
full_labeled.head()

,Document Title,Document ID,Posted Date,Received Date,Document SubType,Post Mark Date,Document Detail,Comment,Comment_DROP,Support_Rule_Change
0,Anonymous public comment,EPA-HQ-OW-2018-0149-6689,2019-04-29,2019-04-11,Public Comment,04/11/2019,https://www.regulations.gov/document?D=EPA-HQ-...,I write to express my opposition to the Enviro...,I write to express my opposition to the Enviro...,0.0
1,Anonymous public comment,EPA-HQ-OW-2018-0149-6688,2019-04-29,2019-04-11,Public Comment,04/11/2019,https://www.regulations.gov/document?D=EPA-HQ-...,I write to express my opposition to the Enviro...,I write to express my opposition to the Enviro...,0.0
2,Anonymous public comment,EPA-HQ-OW-2018-0149-7377,2019-05-01,2019-04-11,Public Comment,04/11/2019,https://www.regulations.gov/document?D=EPA-HQ-...,Please abandon this proposed rollback of the 2...,Please abandon this proposed rollback of the 2...,0.0
3,Anonymous public comment,EPA-HQ-OW-2018-0149-6707,2019-04-29,2019-04-11,Public Comment,04/11/2019,https://www.regulations.gov/document?D=EPA-HQ-...,"My husband and I have two wonderful daughters,...","My husband and I have two wonderful daughters,...",0.0
4,Comment submitted by T. Dahlin,EPA-HQ-OW-2018-0149-7983,2019-05-01,2019-04-12,Public Comment,04/12/2019,https://www.regulations.gov/document?D=EPA-HQ-...,Please do not change the definitions within th...,Please do not change the definitions within th...,0.0


In [11]:
full_labeled.shape

(8187, 10)

In [12]:
# Check that comments align
test = (full_labeled['Comment'] == full_labeled['Comment_DROP']).all()
test

True

In [13]:
# Drop extra comment column
full_labeled.drop('Comment_DROP', axis=1, inplace=True)

full_labeled.head()

,Document Title,Document ID,Posted Date,Received Date,Document SubType,Post Mark Date,Document Detail,Comment,Support_Rule_Change
0,Anonymous public comment,EPA-HQ-OW-2018-0149-6689,2019-04-29,2019-04-11,Public Comment,04/11/2019,https://www.regulations.gov/document?D=EPA-HQ-...,I write to express my opposition to the Enviro...,0.0
1,Anonymous public comment,EPA-HQ-OW-2018-0149-6688,2019-04-29,2019-04-11,Public Comment,04/11/2019,https://www.regulations.gov/document?D=EPA-HQ-...,I write to express my opposition to the Enviro...,0.0
2,Anonymous public comment,EPA-HQ-OW-2018-0149-7377,2019-05-01,2019-04-11,Public Comment,04/11/2019,https://www.regulations.gov/document?D=EPA-HQ-...,Please abandon this proposed rollback of the 2...,0.0
3,Anonymous public comment,EPA-HQ-OW-2018-0149-6707,2019-04-29,2019-04-11,Public Comment,04/11/2019,https://www.regulations.gov/document?D=EPA-HQ-...,"My husband and I have two wonderful daughters,...",0.0
4,Comment submitted by T. Dahlin,EPA-HQ-OW-2018-0149-7983,2019-05-01,2019-04-12,Public Comment,04/12/2019,https://www.regulations.gov/document?D=EPA-HQ-...,Please do not change the definitions within th...,0.0


In [33]:
# Serialize full labeled DF as pickle object
# full_labeled.to_pickle('./Data/full_labeled.pkl')

## Export Docket and Label Data to SQLite Database

There are two versions of the web scraper as noted in the `README.md` file. Instead of running the web scraper a second time to build the database of comments, I used the comment DataFrame created above to populate the tables. The docket and label data are also inserted into their own tables in the database in the code below to have all the information captured in the database file.

In [67]:
# Connect to the database
conn = sqlite3.connect('./Data/comments.db')
cursor = conn.cursor()

In [72]:
comment_table = full_labeled[["Document ID", "Comment"]]
comment_table.columns = ['ID', 'Comment']
comment_table.head()

,ID,Comment
0,EPA-HQ-OW-2018-0149-6689,I write to express my opposition to the Enviro...
1,EPA-HQ-OW-2018-0149-6688,I write to express my opposition to the Enviro...
2,EPA-HQ-OW-2018-0149-7377,Please abandon this proposed rollback of the 2...
3,EPA-HQ-OW-2018-0149-6707,"My husband and I have two wonderful daughters,..."
4,EPA-HQ-OW-2018-0149-7983,Please do not change the definitions within th...


In [76]:
# Create the table and set primary key
create_comments_table_sql = """CREATE TABLE IF NOT EXISTS comments (
                                   ID text PRIMARY KEY,
                                   Comment text
                               );"""

cursor.execute(create_comments_table_sql)
conn.commit()

In [78]:
# Save ID and comment into database table
comment_table.to_sql(name='comments', con=conn, if_exists='append', index=False)
conn.commit()

In [44]:
docket.head()

,Document Title,Document Type,Attachment Count,Document ID,Posted Date,Received Date,Document SubType,Abstract,Status,Post Mark Date,File Type,Number Of Pages,Document Detail
0,Anonymous public comment,PUBLIC SUBMISSIONS,NaN,EPA-HQ-OW-2018-0149-6689,2019-04-29,2019-04-11,Public Comment,NaN,Posted,2019-04-11,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...
1,Anonymous public comment,PUBLIC SUBMISSIONS,NaN,EPA-HQ-OW-2018-0149-6688,2019-04-29,2019-04-11,Public Comment,NaN,Posted,2019-04-11,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...
2,"Comment submitted by Mike Hook, Craighead Coun...",PUBLIC SUBMISSIONS,1.0,EPA-HQ-OW-2018-0149-6690,2019-04-29,2019-04-11,Government Local,NaN,Posted,2019-04-11,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...
3,Anonymous public comment,PUBLIC SUBMISSIONS,NaN,EPA-HQ-OW-2018-0149-7377,2019-05-01,2019-04-11,Public Comment,NaN,Posted,2019-04-11,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...
4,"Comment submitted by Anne Hulick, CT Director,...",PUBLIC SUBMISSIONS,1.0,EPA-HQ-OW-2018-0149-8930,2019-05-04,2019-04-15,Company/Organization Comment,NaN,Posted,2019-04-15,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...


In [45]:
docket.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11411 entries, 0 to 11411
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Document Title    11411 non-null  object        
 1   Document Type     11411 non-null  object        
 2   Attachment Count  3224 non-null   float64       
 3   Document ID       11411 non-null  object        
 4   Posted Date       11411 non-null  datetime64[ns]
 5   Received Date     11411 non-null  datetime64[ns]
 6   Document SubType  11409 non-null  object        
 7   Abstract          72 non-null     object        
 8   Status            11411 non-null  object        
 9   Post Mark Date    11353 non-null  datetime64[ns]
 10  File Type         50 non-null     object        
 11  Number Of Pages   11411 non-null  float64       
 12  Document Detail   11411 non-null  object        
dtypes: datetime64[ns](3), float64(2), object(8)
memory usage: 1.2+ MB


In [80]:
docket.columns = ['doc_title', 'doc_type', 'attachment_count', 'ID',
                  'posted_date', 'received_date', 'doc_subtype',
                  'abstract', 'status', 'post_mark_date', 'file_type',
                  'number_of_pages', 'doc_link']

In [82]:
# Create the table and set primary key
create_docket_table_sql = """CREATE TABLE IF NOT EXISTS docket (
                                 doc_title text,
                                 doc_type text,
                                 attachment_count smallint,
                                 ID text PRIMARY KEY,
                                 posted_date date,
                                 received_date date,
                                 doc_subtype text,
                                 abstract text,
                                 status text,
                                 post_mark_date date,
                                 file_type text,
                                 number_of_pages smallint,
                                 doc_link text
                             );"""

cursor.execute(create_comments_table_sql)
conn.commit()

In [83]:
# Save to database table
docket.to_sql(name='docket', con=conn, if_exists='append', index=False)
conn.commit()

In [86]:
# Create table with ID and labels
labels_table = full_labeled[["Document ID", "Support_Rule_Change"]]
labels_table.columns = ["ID", "Support_Rule_Change"]

In [87]:
labels_table.head()

,ID,Support_Rule_Change
0,EPA-HQ-OW-2018-0149-6689,0.0
1,EPA-HQ-OW-2018-0149-6688,0.0
2,EPA-HQ-OW-2018-0149-7377,0.0
3,EPA-HQ-OW-2018-0149-6707,0.0
4,EPA-HQ-OW-2018-0149-7983,0.0


In [89]:
# Remove unlabeled rows
labels_table.dropna(inplace=True)

In [91]:
labels_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1221 entries, 0 to 8186
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   1221 non-null   object 
 1   Support_Rule_Change  1221 non-null   float64
dtypes: float64(1), object(1)
memory usage: 28.6+ KB


In [92]:
# Create the table and set primary key
create_labels_table_sql = """CREATE TABLE IF NOT EXISTS labels (
                                 ID text PRIMARY KEY,
                                 Support_Rule_Change smallint
                             );"""

cursor.execute(create_labels_table_sql)
conn.commit()

In [93]:
# Save ID and label into database table
labels_table.to_sql(name='labels', con=conn, if_exists='append', index=False)

In [97]:
# Test that the tables saved correctly
# tmp = pd.read_sql("SELECT * FROM labels", conn)
# tmp.head()

In [98]:
# Save all database changes
conn.commit()

# Close connection
conn.close()